In [1]:
%cd /home/x_amime/x_amime/projects/an_explanation_model/

/proj/azizpour-group/users/x_amime/projects/an_explanation_model


/home/x_amime/.conda/envs/xlab10/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import flax
import jax
from matplotlib import pyplot as plt
import numpy as np
import optax 
import tqdm


%load_ext autoreload
%autoreload 2

import pandas as pd
import seaborn as sns
from glob import glob
import PIL

from scipy.stats import ttest_ind_from_stats as ttest
from source.data_manager import single_query_imagenet
from source.project_manager import load_experiment_metadata
from source.data_manager import preprocess, save_axis
from source.model_manager import init_vit_forward

# run this without ? to compute aucs for an experiment
# run this with ? to merge
# run this with ? twice to merge aucs accross experiments
save_metadata_dir = "outputs/metadata/9_?_*/"

imshow_args = {"cmap": "jet"}
dataset_dir = "/proj/azizpour-group/datasets/imagenet"

2024-05-11 10:51:35.574693: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/x_amime/.conda/envs/xlab10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
image,label,path = single_query_imagenet(dataset_dir=dataset_dir,
                                         skip=1,
                                         input_shape=(1,224,224,3),
                                         take=1)

2024-05-11 12:43:08.746216: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [15]:
label

624

In [16]:
import argparse


args = argparse.Namespace(
    save_temp_base_dir="outputs/temp",
    dataset="imagenet",
    input_shape=(1, 224, 224, 3),
)
init_vit_forward(args)
f = args.forward[0]

In [17]:
np.exp(f(image)).argmax()

624